In [14]:
""" 
Try PCA on simulation 2
"""

' \nTry PCA on simulation 2\n'

In [15]:
%matplotlib widget

import sys
sys.path.append("/home/wanxinli/deep_patient/synthetic_exp")

from common import *
from deep_patient.sda import SDA
from math import floor, exp
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import dirichlet
import ot
from numpy.random import poisson
import pandas as pd
from random import randint
from scipy import sparse
from sklearn import linear_model
from sklearn import metrics
from sklearn import svm
from sklearn.decomposition import PCA

base_dir = "/home/wanxinli/deep_patient"

In [37]:
""" 
Simulation scheme
"""

def simulate(D, d_1, d_2, num_patient):
    """ 
    Simulate features and labels for domain 1 and domain 2
    :param int D:  total number of features
    :param int d_1: number of features with higher frequency in domain 1
    :param int d_2: number of features with higher frequency in domain 2
    :param int num_patient: number of patients in each domain

    Variables in the implementation are consistent with the variables in the scheme


    :return
        list[list[int]] domain 1 feature vectors
        list[int] domain 1 labels
        list[list[int]] domain 2 feature vectors
        list[int] domain 2 labels
    """

    d_1 = randint(0, floor(0.25*D))
    d_2 = randint(0, floor(0.25*D))
    delta_1 = np.random.choice(size = d_1, a = range(1, D+1), replace=False)
    remaining_set = list(set(list(range(1, D+1)))-set(delta_1))
    delta_2 = np.random.choice(size = d_1, a = remaining_set, replace=False)
    
    # We set the proportions of d_1 codes, d_2 codes, and (D-d_1-d_2) codes to be 2:1:1.5
    unit_1 = 1/(0.5*d_1-0.5*d_2+1.5*D)
    alpha_1 = [2*unit_1]*d_1
    alpha_1.extend([unit_1]*d_2)
    alpha_1.extend([1.5*unit_1]*(D-d_1-d_2))

    # We set the proportions of d_1 codes, d_2 codes, and (D-d_1-d_2) codes to be 1:2:1.5
    unit_2 = 1/(-0.5*d_1+0.5*d_2+1.5*D)
    alpha_2 = [2*unit_2]*d_1
    alpha_2.extend([unit_2]*d_2)
    alpha_2.extend([1.5*unit_2]*(D-d_1-d_2))    

    def gen_feature_vector_label(alpha):
        """ 
        Generate feature vectors and labels
        :param list[float] alpha: concentration parameteres for the dirichlet distribution
        """

        def sigmoid(x):
            return 1 / (1 + exp(-x))

        rho = dirichlet(alpha=alpha, size=1)[0]
        W = np.random.normal(size=D)
        W = [max(0, W_k) for W_k in W] # only sample positive weights
        X = []
        Y = []
        b = 0
        all_sum = []

        for _ in range(num_patient):
            X_i = np.random.multinomial(3*len(rho), rho)
            for k in range(len(X_i)):
                if X_i[k] > 0:
                    X_i[k] = 1 # dominant effect
            X.append(X_i)
            lambda_i= np.sum(np.multiply(W, X_i))
            Y_i = poisson(lam = lambda_i)
            Y.append(Y_i)
      
        return X, Y, W, b
    
    def feature_vector_to_feature(feature_vectors):
        """ 
        Convert feature vectors to features
        :param list[list[int]]: feature vectors consisting of indicators

        Returns
            - features consisting of actual codes
        """
        features = []
        for feature_vector in feature_vectors:
            features.append([i for i, e in enumerate(feature_vector) if e != 0])
        return features
    

    feature_vector_1, label_1, W_1, b_1 = gen_feature_vector_label(alpha_1)
    feature_vector_2, label_2, W_2, b_2 = gen_feature_vector_label(alpha_2)
    return feature_vector_1, np.array(label_1), feature_vector_2, np.array(label_2)



In [38]:
def simulation_wrapper():
    num_patient = 1000
    D = 20
    d_1 = 5
    d_2 = 5
    return simulate(D, d_1, d_2, num_patient)

In [57]:
""" 
Fit PCA
"""

target_features, target_labels, source_features, source_labels = simulation_wrapper()
target_pca = PCA(n_components=3)
target_reps = target_pca.fit_transform(target_features)
source_pca = PCA(n_components=3)
source_reps = source_pca.fit_transform(source_features)
reconstrcuted_source_features = source_pca.inverse_transform(source_reps)


In [58]:
source_features

[array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 

In [59]:
reconstrcuted_source_features


array([[ 1.00000000e+00, -1.02907346e-16,  1.00657301e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  8.32455571e-18,  9.84849355e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  1.73945340e-17,  9.86467988e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.00000000e+00,  1.73945340e-17,  9.86467988e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  8.32455571e-18,  9.84849355e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  8.32455571e-18,  9.84849355e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [60]:
""" 
Sinkhorn transport source reps onto domain domain
"""
trans_source_reps =  trans_source2target(source_reps, target_reps)

In [61]:
""" 
Fit models 
"""

target_model = linear_model.LinearRegression()
target_model.fit(target_reps, target_labels)

# calculate the stats
target_pred_labels = target_model.predict(target_reps)


In [62]:
# print(target_mae, target_mse, target_rmse)
# print(source_mae, source_mse, source_rmse)
# print(trans_source_mae, trans_source_mse, trans_source_rmse)

In [63]:
target_reps

array([[-0.18669053, -0.0428011 , -0.08848433],
       [-0.18669053, -0.0428011 , -0.08848433],
       [-0.18669053, -0.0428011 , -0.08848433],
       ...,
       [-0.18669053, -0.0428011 , -0.08848433],
       [-0.18669053, -0.0428011 , -0.08848433],
       [-0.23310434, -0.38667336,  0.84406964]])

In [64]:
# vis_emb_dim2_ordered(target_reps, target_labels, source_reps, source_labels, trans_source_reps, target_model)

In [65]:
""" 
Calculate stats 
"""
target_mae, target_mse, target_rmse, source_mae, source_mse, source_rmse,\
        trans_source_mae, trans_source_mse, trans_source_rmse = \
    cal_stats_cts(target_reps, target_labels, source_reps, source_labels, \
    trans_source_reps, linear_model.LinearRegression)
print(target_mae, target_mse, target_rmse)
print(source_mae, source_mse, source_rmse)
print(trans_source_mae, trans_source_mse, trans_source_rmse)

0.9775957754404182 1.7907450025636371 1.338187207592285
2.1923443108193625 7.544745910989338 2.7467700870275507
2.1691761899459063 7.386750337239563 2.7178576742058373
